# Getting the data from the db about plant optimal ranges

In [15]:
import requests
import pandas as pd 

In [4]:
base_url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/"

In [6]:
plant_data = requests.get(base_url + "plants/").text
plant_json = json.loads(plant_data)
plant_json

{'documents': [{'name': "projects/mimirhome-app/databases/(default)/documents/plants/acanthocereus tetragonus 'fairy castles'",
   'fields': {'display_pid': {'stringValue': "Acanthocereus tetragonus 'Fairy Castles'"},
    'image': {'stringValue': 'http://pkb.resource.huahuacaocao.com/YWNhbnRob2NlcmV1cyB0ZXRyYWdvbnVzICdmYWlyeSBjYXN0bGVzJy5qcGc=?imageView2/1/w/%d/h/%d'},
    'basic': {'mapValue': {'fields': {'floral_language': {'stringValue': "Acanthocereus tetragonus 'Fairy Castles'\nFloriography: strong, kind, persistent.\nSucculent stem of the Acanthocereus tetragonus 'Fairy Castles' is oval and palm shaped, thick and juicy. Leaves reduce to long needle spines. Because it is almost care free, Acanthocereus tetragonus 'Fairy Castles' is very popular and could be found on desktop, tea table, bedroom and office.\nAcanthocereus tetragonus 'Fairy Castles' requires direct bright sunlight for growth. It blooms in certain conditions. Flower color could be light yellow, orange or pink."},
    

In [7]:
len(plant_json['documents'])

20

## doing it for one species 

In [9]:
plant_json["nextPageToken"]

'AFTOeJxCaMmLo2pCW7Iy37c_k5E5BPWzsK2GdVuz7zT--P6Jzxg7BNfsIKafMvSYI9wx9ze7VFj9oy8CPb6qfsOe7Qym-pxLS1XJHo6M2qd8nzmNTzyPifXa7Wrq7QR3V_TWfBWz4-YP69RJc9s_'

In [30]:
plant_json['documents'][0]['fields']['parameter']['mapValue']['fields']

{'max_env_humid': {'integerValue': '80'},
 'min_light_mmol': {'integerValue': '3500'},
 'max_light_lux': {'integerValue': '100000'},
 'min_light_lux': {'integerValue': '3000'},
 'min_env_humid': {'integerValue': '15'},
 'min_temp': {'integerValue': '5'},
 'max_soil_moist': {'integerValue': '50'},
 'max_soil_ec': {'integerValue': '1000'},
 'min_soil_ec': {'integerValue': '100'},
 'max_light_mmol': {'integerValue': '8700'},
 'min_soil_moist': {'integerValue': '7'},
 'max_temp': {'integerValue': '35'}}

In [39]:
plant_json['documents'][0]['fields']['display_pid']['stringValue']

"Acanthocereus tetragonus 'Fairy Castles'"

In [33]:
df = pd.DataFrame(plant_json['documents'][0]['fields']['parameter']['mapValue']['fields'])

In [34]:
df 

,max_env_humid,min_light_mmol,max_light_lux,min_light_lux,min_env_humid,min_temp,max_soil_moist,max_soil_ec,min_soil_ec,max_light_mmol,min_soil_moist,max_temp
integerValue,80,3500,100000,3000,15,5,50,1000,100,8700,7,35


In [40]:
df['name'] = plant_json['documents'][0]['fields']['display_pid']['stringValue']

In [41]:
df 

,max_env_humid,min_light_mmol,max_light_lux,min_light_lux,min_env_humid,min_temp,max_soil_moist,max_soil_ec,min_soil_ec,max_light_mmol,min_soil_moist,max_temp,name
integerValue,80,3500,100000,3000,15,5,50,1000,100,8700,7,35,Acanthocereus tetragonus 'Fairy Castles'


In [46]:
df.columns

Index(['max_env_humid', 'min_light_mmol', 'max_light_lux', 'min_light_lux',
       'min_env_humid', 'min_temp', 'max_soil_moist', 'max_soil_ec',
       'min_soil_ec', 'max_light_mmol', 'min_soil_moist', 'max_temp', 'name'],
      dtype='object')

In [47]:
cols = ['name', 'min_env_humid', 'max_env_humid', 'min_light_lux', 'max_light_lux','min_temp', 'max_temp']

In [50]:
df.drop(['min_light_mmol', 'max_soil_moist', 'max_soil_ec', 'min_soil_ec', 'max_light_mmol', 'min_soil_moist'], axis = 1, inplace = True)

In [52]:
df = df[cols] 

In [53]:
df 

,name,min_env_humid,max_env_humid,min_light_lux,max_light_lux,min_temp,max_temp
integerValue,Acanthocereus tetragonus 'Fairy Castles',15,80,3000,100000,5,35


## repeating this for all the species

In [69]:
cols = ['name', 'min_env_humid', 'max_env_humid', 'min_light_lux', 'max_light_lux','min_temp', 'max_temp']
initial_df = pd.DataFrame(columns = cols)

In [70]:
url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/plants"
page_token = plant_json["nextPageToken"]
data2 = requests.get(url + f"?pageToken={page_token}").text
plant_json2 = json.loads(data2)

In [71]:
for i in range(len(plant_json2['documents'])):
    df = pd.DataFrame(plant_json['documents'][i]['fields']['parameter']['mapValue']['fields'])
    df['name'] = plant_json['documents'][i]['fields']['display_pid']['stringValue']
    df.drop(['min_light_mmol', 'max_soil_moist', 'max_soil_ec', 'min_soil_ec', 'max_light_mmol', 'min_soil_moist'], axis = 1, inplace = True)
    df = df[cols] 
    initial_df = initial_df.append(df)

In [72]:
initial_df

,name,min_env_humid,max_env_humid,min_light_lux,max_light_lux,min_temp,max_temp
integerValue,Acanthocereus tetragonus 'Fairy Castles',15,80,3000,100000,5,35
integerValue,Adi violet wax,30,80,3000,50000,5,32
integerValue,Aechmea chantinii 'Samuri',30,85,800,27000,10,37
integerValue,Aechmea fulgens,30,85,800,27000,10,37
integerValue,Aeonium arboreum,15,80,3000,80000,5,35
integerValue,Aeonium arboreum 'Atropureum',15,80,2800,80000,6,35
integerValue,Aeonium decorum f. variegata,15,80,3800,85000,6,35
integerValue,Aeonium sedifolium,15,80,3200,80000,5,35
integerValue,Aeonium urbicum,15,80,3000,80000,5,35
integerValue,Agastache foeniculum,30,80,4000,60000,5,35


In [73]:
# getting all json files into df ---- this is the right thing!!!!!!!
url = "https://firestore.googleapis.com/v1/projects/mimirhome-app/databases/(default)/documents/plants"
plant_data = requests.get(url).text
cols = ['name', 'min_env_humid', 'max_env_humid', 'min_light_lux', 'max_light_lux','min_temp', 'max_temp']
#initial_df = pd.DataFrame(columns = cols)

while plant_json["nextPageToken"]:
    page_token = plant_json["nextPageToken"]
    data = requests.get(url + f"?pageToken={page_token}").text
    plant_json = json.loads(data)
    for i in range(len(plant_json['documents'])):
        df = pd.DataFrame(plant_json['documents'][i]['fields']['parameter']['mapValue']['fields'])
        df['name'] = plant_json['documents'][i]['fields']['display_pid']['stringValue']
        df.drop(['min_light_mmol', 'max_soil_moist', 'max_soil_ec', 'min_soil_ec', 'max_light_mmol', 'min_soil_moist'], axis = 1, inplace = True)
        df = df[cols] 
        initial_df = initial_df.append(df)

KeyError: 'nextPageToken'

In [74]:
final_df = initial_df.drop_duplicates() 
final_df 

,name,min_env_humid,max_env_humid,min_light_lux,max_light_lux,min_temp,max_temp
integerValue,Acanthocereus tetragonus 'Fairy Castles',15,80,3000,100000,5,35
integerValue,Adi violet wax,30,80,3000,50000,5,32
integerValue,Aechmea chantinii 'Samuri',30,85,800,27000,10,37
integerValue,Aechmea fulgens,30,85,800,27000,10,37
integerValue,Aeonium arboreum,15,80,3000,80000,5,35
...,...,...,...,...,...,...,...
integerValue,Turbinicarpus pseudopectinatus,15,80,2000,70000,8,35
integerValue,Vriesea poelmannii,30,85,800,27000,10,37
integerValue,Wax purple,30,80,3000,50000,5,32
integerValue,Yellow Crocus spp.,30,80,2500,30000,6,35


In [75]:
final_df.to_csv('plant_data.csv', index=False)